In [1]:
import numpy as np 
import pandas as pd
import copy
from tqdm.notebook import tqdm
import plotly.express as px
from scipy import stats
import scipy.constants as const
import matplotlib.pyplot as plt

### Example from Jaguar Calculator

In [2]:
class Reliability(object):
      def __init__(
            self,
            MRM_RTH_SEL = 0
      ):
            self.MRM_RTH_SEL = MRM_RTH_SEL
            if MRM_RTH_SEL == 1:
                  self.MRM_MAIN_HTR_RTH = 10972 # degC/W
            else:
                  self.MRM_MAIN_HTR_RTH = 9800 # degC/W
            self.CRR_HTR_RTH = 10068 # degC/W

            self.MRM_MAIN_HTR_PFSR = 22e-3 # W
            self.CRR_HTR_PFSR = 18.4e-3 # W

            self.FSR_GHZ = 1800 # GHz

            self.MRM_dT_FSR = 156 # degC
            self.CRR_dT_FSR = 162.4 # degC

            self.COLD_CHIP_TEMP = 10 # degC
            self.MEDIAN_CHIP_TEMP = 65 #degC
            self.HOT_CHIP_TEMP = 95 # degC

            self.MAX_MRM_MAIN_HTR_TEMP = 10
            self.MAX_CRR_HTR_TEMP = 10
            self.LIFETIME_YEARS = np.logspace(-1,1,101)
            
            self.FIRST_ATTEMPT = False
            
      def update_state(self):
            self.MRM_MAIN_HTR_FREQ = 6/9 * self.FSR_GHZ # GHz
            self.CRR_HTR_FREQ = 8/9 * self.FSR_GHZ # GHz

            self.MRM_MAIN_HTR_CHIP_TEMP = (self.HOT_CHIP_TEMP - self.COLD_CHIP_TEMP)/self.MRM_dT_FSR*self.FSR_GHZ# degC
            self.CRR_HTR_CHIP_TEMP = (self.HOT_CHIP_TEMP - self.COLD_CHIP_TEMP)/self.CRR_dT_FSR*self.FSR_GHZ # degC

            self.MEDIAN_MRM_MAIN_HTR_CHIP_TEMP = (self.HOT_CHIP_TEMP - self.MEDIAN_CHIP_TEMP)/self.MRM_dT_FSR*self.FSR_GHZ# degC
            self.MEDIAN_CRR_HTR_CHIP_TEMP = (self.HOT_CHIP_TEMP - self.MEDIAN_CHIP_TEMP)/self.CRR_dT_FSR*self.FSR_GHZ # degC

            
      def algo_htr_temp(self, mrm_main_htr_freq, crr_htr_freq):
            self.cold_chip_mrm_main_htr_temp = self.COLD_CHIP_TEMP + self.MRM_MAIN_HTR_PFSR*self.MRM_MAIN_HTR_RTH*(self.MRM_MAIN_HTR_FREQ/self.FSR_GHZ + self.MRM_MAIN_HTR_CHIP_TEMP/self.FSR_GHZ) # degC
            self.cold_chip_crr_htr_temp = self.COLD_CHIP_TEMP + self.CRR_HTR_PFSR*self.CRR_HTR_RTH*(self.CRR_HTR_FREQ/self.FSR_GHZ + self.CRR_HTR_CHIP_TEMP/self.FSR_GHZ) # degC

            self.median_chip_mrm_main_htr_temp = self.MEDIAN_CHIP_TEMP + self.MRM_MAIN_HTR_PFSR*self.MRM_MAIN_HTR_RTH*(self.MRM_MAIN_HTR_FREQ/self.FSR_GHZ + self.MEDIAN_MRM_MAIN_HTR_CHIP_TEMP/self.FSR_GHZ) # degC 
            self.median_chip_crr_htr_temp = self.MEDIAN_CHIP_TEMP + self.CRR_HTR_PFSR*self.CRR_HTR_RTH*(self.CRR_HTR_FREQ/self.FSR_GHZ + self.MEDIAN_CRR_HTR_CHIP_TEMP/self.FSR_GHZ) # degC

            self.hot_chip_mrm_main_htr_temp = self.HOT_CHIP_TEMP + self.MRM_MAIN_HTR_PFSR*self.MRM_MAIN_HTR_RTH*(self.MRM_MAIN_HTR_FREQ/self.FSR_GHZ) # degC
            self.hot_chip_crr_htr_temp = self.HOT_CHIP_TEMP + self.CRR_HTR_PFSR*self.CRR_HTR_RTH*(self.CRR_HTR_FREQ/self.FSR_GHZ) # degC
            
            ## Normalizing the heater frequency
            mrm_main_htr_freq = mrm_main_htr_freq%self.FSR_GHZ
            crr_htr_freq = crr_htr_freq%self.FSR_GHZ

            ## Calculating heater temperature with simple algorithm
            hot_chip_mrm_main_htr_temp = self.HOT_CHIP_TEMP + self.MRM_MAIN_HTR_PFSR*self.MRM_MAIN_HTR_RTH*(mrm_main_htr_freq/self.FSR_GHZ) # degC
            hot_chip_crr_htr_temp = self.HOT_CHIP_TEMP + self.CRR_HTR_PFSR*self.CRR_HTR_RTH*(crr_htr_freq/self.FSR_GHZ) # degC

            median_chip_mrm_main_htr_temp = self.MEDIAN_CHIP_TEMP + self.MRM_MAIN_HTR_PFSR*self.MRM_MAIN_HTR_RTH*(mrm_main_htr_freq/self.FSR_GHZ + self.MEDIAN_MRM_MAIN_HTR_CHIP_TEMP/self.FSR_GHZ) # degC
            median_chip_crr_htr_temp = self.MEDIAN_CHIP_TEMP + self.CRR_HTR_PFSR*self.CRR_HTR_RTH*(crr_htr_freq/self.FSR_GHZ + self.MEDIAN_CRR_HTR_CHIP_TEMP/self.FSR_GHZ) # degC

            cold_chip_mrm_main_htr_temp = self.COLD_CHIP_TEMP + self.MRM_MAIN_HTR_PFSR*self.MRM_MAIN_HTR_RTH*(mrm_main_htr_freq/self.FSR_GHZ + self.MRM_MAIN_HTR_CHIP_TEMP/self.FSR_GHZ) # degC
            cold_chip_crr_htr_temp = self.COLD_CHIP_TEMP + self.CRR_HTR_PFSR*self.CRR_HTR_RTH*(crr_htr_freq/self.FSR_GHZ + self.CRR_HTR_CHIP_TEMP/self.FSR_GHZ) # degC

            self.mc_htr_temp = pd.DataFrame({
                  'MRM_HEAT_FREQ': mrm_main_htr_freq, 
                  'HOT_CHIP_MRM_HEATER_TEMP': hot_chip_mrm_main_htr_temp,
                  'MEDIAN_CHIP_MRM_HEATER_TEMP': median_chip_mrm_main_htr_temp,
                  'COLD_CHIP_MRM_HEATER_TEMP': cold_chip_mrm_main_htr_temp,       
                  
                  'CRR_HEAT_FREQ': crr_htr_freq,     
                  'HOT_CHIP_CRR_HEATER_TEMP': hot_chip_crr_htr_temp,
                  'MEDIAN_CHIP_CRR_HEATER_TEMP': median_chip_crr_htr_temp,
                  'COLD_CHIP_CRR_HEATER_TEMP': cold_chip_crr_htr_temp,  
            })
            
      def htr_reliability(self):
            for idd_string, string in enumerate(['MEDIAN_CHIP_MRM_HEATER_TEMP', 'MEDIAN_CHIP_CRR_HEATER_TEMP']):
                  stats_ecdf = stats.ecdf(self.mc_htr_temp[string].values)
                  quantiles = stats_ecdf.cdf.quantiles
                  pdf = stats_ecdf.cdf.probabilities
                  T_range = quantiles + const.zero_Celsius
                  kTinv_range = const.eV / T_range / const.k

                  Ea_fit = np.array([1.966256206948785, np.log(8.88893341469434e-15)])

                  fit_factor = 22

                  mttf_range = np.exp(Ea_fit[1]) * np.exp(Ea_fit[0] * kTinv_range)*fit_factor
                  sigma = 0.4

                  size = 101

                  frac_years = np.zeros((len(T_range), len(self.LIFETIME_YEARS)))

                  for idx in range(len(T_range)):
                        for idy in range(len(self.LIFETIME_YEARS)):
                              s = stats.lognorm(s=sigma, scale=mttf_range[idx])
                              frac_years[idx, idy] = s.pdf(self.LIFETIME_YEARS[idy]*365*24)
                  
                  if string == 'MEDIAN_CHIP_MRM_HEATER_TEMP':
                        if self.MRM_RTH_SEL == 1:
                              device = 'MRM_Rth_11kCperW'
                        else:
                              device = 'MRM_Rth_9p8kCperW'
                  else:
                        device = 'CRR_Rth_10kCperW'
                  
                  lifetime = np.cumsum(frac_years.T @ pdf) * (256*4) * 1e6
                  temp = pd.DataFrame({
                        'Device': device,
                        'Time': self.LIFETIME_YEARS,
                        'Lifetime': lifetime,
                        'Tj_max': self.HOT_CHIP_TEMP,
                        'Top_median': self.MEDIAN_CHIP_TEMP,
                  })
                  if self.FIRST_ATTEMPT == False:
                        self.mc_htr_reliability = temp.copy() 
                        self.FIRST_ATTEMPT = True
                  else:
                        self.mc_htr_reliability = pd.concat([self.mc_htr_reliability, temp.copy()])
      
      def plot_htr_temperature(self, mrm_main_htr_freq, crr_htr_freq):
            for self.HOT_CHIP_TEMP in [90, 95, 100, 105]:
                  self.update_state()
                  self.algo_htr_temp(mrm_main_htr_freq, crr_htr_freq)
                  
            fig = px.histogram(self.mc_htr_temp, x=['MEDIAN_CHIP_MRM_HEATER_TEMP', 'MEDIAN_CHIP_CRR_HEATER_TEMP'], marginal="histogram", histnorm="probability density", width=700, height=300)
            fig.update_layout(xaxis_title="Heater temperature (degC)", yaxis_title="PDF", title="MRM and CRR heating using simple algorithm")
            fig.update_layout(
                  xaxis = dict(
                        tickmode = 'array',
                        tickvals = list(np.arange(100, 340, 20)),
                  ),
                  xaxis_range = [100, 320]
            )
            if self.MRM_RTH_SEL == 1:
                  fig.write_image(f'./htr_temp_Rth_11kCperW.png', scale=10)
            else:
                  fig.write_image(f'./htr_temp_Rth_9p8kCperW.png', scale=10)
                  
      
      def plot_htr_reliability(self, mrm_main_htr_freq, crr_htr_freq):
            for self.HOT_CHIP_TEMP in [90, 95, 100, 105]:
                  self.update_state()
                  self.algo_htr_temp(mrm_main_htr_freq, crr_htr_freq)
                  self.htr_reliability()
            for devices in self.mc_htr_reliability['Device'].unique():
                  fig = px.line(self.mc_htr_reliability[self.mc_htr_reliability['Device'] == devices], x='Time', y='Lifetime', color='Tj_max', log_x=False, log_y=True, width=500, height=300)
                  fig.update_layout(xaxis_title="Lifetime [years]", yaxis_title="Chip level failures [dpm]", showlegend=True, title=f'Device: {devices}')
                  fig.update_layout(
                        xaxis = dict(
                              tickmode = 'array',
                              tickvals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
                        ),
                        xaxis_range = [1, 10],
                        yaxis = dict(
                              tickmode = 'array',
                              tickvals = [1e-2, 1e-1, 1e0, 1e1, 5e1, 1e2, 5e2, 1e3]
                        ),
                        yaxis_range = [-2, 3],
                  )
                  if self.MRM_RTH_SEL == 1:
                        fig.write_image(f'./{devices}_reliability_Rth_11kCperW.png', scale=10)
                  else:
                        fig.write_image(f'./{devices}_reliability_Rth_9p8kCperW.png', scale=10)

In [3]:
mc_results_mrm_simple_algo = pd.read_csv('trial_results_mrm_simple_algo.csv')
mc_results_crr_simple_algo = pd.read_csv('trial_results_crr_simple_algo.csv')

for idd in range(8):
      if idd == 0:
        mrm_main_htr_freq_simple_algo = np.array(mc_results_mrm_simple_algo[str(idd)]).reshape(-1) * -1 + 400
        crr_htr_freq_simple_algo = np.array(mc_results_crr_simple_algo[str(idd)]).reshape(-1) * -1 + 400
      else:
        mrm_main_htr_freq_simple_algo = np.concatenate((mrm_main_htr_freq_simple_algo, np.array(mc_results_mrm_simple_algo[str(idd)]).reshape(-1)* -1 + 400))
        crr_htr_freq_simple_algo = np.concatenate((crr_htr_freq_simple_algo, np.array(mc_results_crr_simple_algo[str(idd)]).reshape(-1)* -1 + 400))

In [4]:
reliability_compute = Reliability()
reliability_compute.plot_htr_temperature(mrm_main_htr_freq_simple_algo, crr_htr_freq_simple_algo)
reliability_compute.plot_htr_reliability(mrm_main_htr_freq_simple_algo, crr_htr_freq_simple_algo)